In [1]:
import nltk
import spacy
import gensim
import numpy as np
import pandas as pd
from nltk.stem import *
from nltk.stem.porter import *
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess, deaccent
from gensim.models.coherencemodel import CoherenceModel

import pyLDAvis.gensim

In [2]:
def verificar_palavra_entidade_loc(palavra, entidades_loc):
	"""
	Verifica se a palavra é uma entidade de localização.

	Parâmetros:
	----------
	palavra : String
		- Palavra a ser verificada.
	entidades_loc : List
		- Lista de entidades de localizações reconhecidas pelo Spacy.

	Retorno:
	----------
	True : Caso a palavra seja uma entidade de localização.\n
	False : Caso a palavra não seja uma entidade de localização.
	"""
    
	for e in entidades_loc:
		if (e.text.lower() == palavra.lower()):
			return True

	return False

# Configurando bibliotecas e variaveis globais.
stemmer = PorterStemmer()
nlp = spacy.load("pt_core_news_sm")

nlp.Defaults.stop_words |= {"tudo", "coisa", "toda", "tava", "pessoal", "dessa", "resolvido", "aqui", "gente", "tá", "né", "calendário", "jpb", "agora", "voltar", "lá", "hoje", "aí", "ainda", "então", "vai", "porque", "moradores", "fazer", "rua", "bairro", "prefeitura", "todo", "vamos", "problema", "fica", "ver", "tô"}
stop_words_spacy = nlp.Defaults.stop_words

def lematizacao(palavra):
    """
	Realiza a lematização de uma palavra.

	Parâmetro:
	----------
	palavra : String
		- Palavra que irá sofrer a lematização.

	Retorno:
	----------
	palavra : String
		- Palavra lematizada.
	"""
    return stemmer.stem(WordNetLemmatizer().lemmatize(palavra, pos="v"))

allowed_postags = ['NOUN', 'ADJ', 'PRON']
def pre_processamento(texto):
    """
	Realiza o pré-processamento de um texto:
		- Remove Stop Words.
		- Remove palavras que são entidades de localizações.
		- Colocar as palavras para caixa baixa.
		- Realiza a lematização das palavras.
		- Apenas palavras que são: substantivos, adjetivos e pronomes.

	Parâmetro:
	----------
	texto : String
		- Texto que irá sofrer o pré-processamento.
	titulo: String
		- Titulo do texto.

	Retorno:
	----------
	doc_out : List
		- Lista de palavras que passaram pelo pré-processamento.
	"""
    doc_out = []
    doc = nlp(texto)
    entidades_loc = [entidade for entidade in doc.ents if entidade.label_ == "LOC"]
    for token in doc:
        if (token.text not in stop_words_spacy and len(token.text) > 3 and token.pos_ in allowed_postags and not verificar_palavra_entidade_loc(token.text, entidades_loc)):
            doc_out.append(lematizacao(token.text))

    return doc_out

In [3]:
# CONFIGURAÇÕES DE BIBLIOTECAS.
np.random.seed(2018)
nltk.download('wordnet')
nlp = spacy.load('pt_core_news_sm')

# CARREGANDO OS DADOS.
dados = pd.read_csv("./textos_limpos.csv")
dados.drop_duplicates(['texto'], inplace=True)
textos = dados['texto']
# print(textos[:5])

nlp.Defaults.stop_words |= {"volta", "hora", "tapa", "tudo", "coisa", "toda", "tava", "pessoal", "dessa", "resolvido", "aqui", "gente", "tá", "né", "calendário", "jpb", "agora", "voltar", "lá", "hoje", "aí", "ainda", "então", "vai", "porque", "moradores", "fazer", "rua", "bairro", "prefeitura", "todo", "vamos", "problema", "fica", "ver", "tô"}
stop_words_spacy = nlp.Defaults.stop_words

from nltk import word_tokenize, pos_tag

# PRÉ-PROCESSAMENTO DOS DADOS.

# Chamando a função de pré-processamento para cada texto.
processed_docs = dados['texto'].map(lambda texto: texto.split())
print(processed_docs[:10])

# Criando dicionário de palavras.
dictionary = gensim.corpora.Dictionary(processed_docs)

# Gensim Filter Extremes
# Filtrar tokens que aparecem em menos de 15 documentos
# ou em mais de 0.5 documentos(fração do tamanho total do corpus)
# Após essas duas etapas, mantenha apenas os 100000
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# Bag of Words(Saco de Palavras).
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Usando TF-IDF.
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

# Criando e treinando o modelo.
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=10)
# lda_model_tfidf.save("./modelo/meu_lda_model")

[nltk_data] Downloading package wordnet to /home/rick/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    [volta, equip, capit, jeito, mato, lixo, condi...
1    [feliz, construção, crech, capit, inauguração,...
2    [hora, correspondência, polícia, pizza, tarefa...
3    [situação, prejuízo, bolso, risco, vida, burac...
4    [algun, região, unidad, básica, saúd, obra, eq...
5    [simpl, muro, escola, cobrança, distrito, falt...
6    [casa, rosemira, contato, poda, árvore, fiação...
7    [tempinho, preciso, fevereiro, estrutura, cole...
8    [dose, dupla, mercado, notícia, ruim, melhorad...
9    [hora, feliz, carinho, caso, pai, aluno, refor...
Name: texto, dtype: object


In [4]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary=lda_model_tfidf.id2word)
vis

/home/rick/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.045473 -0.059754       1        1  48.729221
2      0.046807  0.063127       2        1  36.954628
1     -0.091410  0.002657       3        1  10.097404
3     -0.000870 -0.006030       4        1   4.218753, topic_info=    Category       Freq       Term      Total  loglift  logprob
276  Default  11.000000    estrada  11.000000  30.0000  30.0000
526  Default  14.000000  velocidad  14.000000  29.0000  29.0000
93   Default  13.000000      faixa  13.000000  28.0000  28.0000
146  Default  35.000000     escola  35.000000  27.0000  27.0000
275  Default  39.000000     esgoto  39.000000  26.0000  26.0000
..       ...        ...        ...        ...      ...      ...
64    Topic4   0.400620      carro  23.605701  -0.9106  -6.2971
153   Topic4   0.372717       medo  10.764426  -0.1976  -6.3693
53    Topic4   0.368625  município   9.532658  -0.0871  -6.3804
146   Topic4   0.400240     escola  35.597816  -1.3223  -6.2981
26    Topic4   0.382171     pessoa  19.349466  -0.7589  -6.3443

[239 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
591       1  0.845659  abastecimento
299       1  0.783374         aberto
433       1  0.192108       academia
433       2  0.768431       academia
381       1  0.619788         acesso
...     ...       ...            ...
84        1  0.954358           água
84        2  0.047718           água
245       1  0.357407          ônibu
245       2  0.071481          ônibu
245       3  0.500369          ônibu

[402 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])